<a href="https://colab.research.google.com/github/christiejibaraki/CUREBench/blob/finetune-gpt-oss-20b/notebooks/3-finetune_gpt_oss_20b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Run CUREBench eval
- run prediction
- run eval

In [5]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [6]:
import os
import sys
import os.path

In [7]:
os.environ['BNB_CUDA_VERSION'] = '125'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

## Setup
- Clone forked CUREBench repo onto local `content` folder (this is not persistent)
- Create virtual environment and install packages
  - Package installation takes about **5 min** ⏰
- Add virtual environment's site-packages to notebook's system path

In [1]:
!git clone -b finetune-gpt-oss-20b https://github.com/christiejibaraki/CUREBench.git

Cloning into 'CUREBench'...
remote: Enumerating objects: 255, done.
remote: Counting objects: 100% (167/167), done.
remote: Compressing objects: 100% (117/117), done.
remote: Total 255 (delta 98), reused 94 (delta 50), pack-reused 88 (from 2)
Receiving objects: 100% (255/255), 3.01 MiB | 13.59 MiB/s, done.
Resolving deltas: 100% (127/127), done.


In [2]:
%cd CUREBench

/content/CUREBench


In [3]:
!git pull

Already up to date.


In [4]:
!pip install virtualenv

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 10.1 MB/s eta 0:00:00


In [8]:
!virtualenv env

created virtual environment CPython3.12.12.final.0-64 in 987ms
  creator CPython3Posix(dest=/content/CUREBench/env, clear=False, no_vcs_ignore=False, global=False)
  seeder FromAppData(download=False, pip=bundle, via=copy, app_data_dir=/root/.local/share/virtualenv)
    added seed packages: pip==25.2
  activators BashActivator,CShellActivator,FishActivator,NushellActivator,PowerShellActivator,PythonActivator


In [9]:
!./env/bin/pip list

Package Version
------- -------
pip     25.2


In [10]:
!./env/bin/pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.9/16.9 MB 12.5 MB/s  0:00:01
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 567.4/567.4 kB 31.6 MB/s  0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 171.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 82.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 137.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 56.4 MB/s  0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 16.2 MB/s  0:00:22
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 117.7 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1

In [11]:
env_path = './env/lib/python3.12/site-packages'
sys.path.append(env_path)

### load dataset

In [12]:
from datasets import load_dataset

ds = load_dataset("FreedomIntelligence/medical-o1-reasoning-SFT", "en")

README.md: 0.00B [00:00, ?B/s]

medical_o1_sft.json:   0%|          | 0.00/58.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/19704 [00:00<?, ? examples/s]

In [13]:
ds

DatasetDict({
    train: Dataset({
        features: ['Question', 'Complex_CoT', 'Response'],
        num_rows: 19704
    })
})

In [19]:
train_data = ds['train']

In [21]:
train_data[0]

{'Question': 'Given the symptoms of sudden weakness in the left arm and leg, recent long-distance travel, and the presence of swollen and tender right lower leg, what specific cardiac abnormality is most likely to be found upon further evaluation that could explain these findings?',
 'Complex_CoT': "Okay, let's see what's going on here. We've got sudden weakness in the person's left arm and leg - and that screams something neuro-related, maybe a stroke?\n\nBut wait, there's more. The right lower leg is swollen and tender, which is like waving a big flag for deep vein thrombosis, especially after a long flight or sitting around a lot.\n\nSo, now I'm thinking, how could a clot in the leg end up causing issues like weakness or stroke symptoms?\n\nOh, right! There's this thing called a paradoxical embolism. It can happen if there's some kind of short circuit in the heart - like a hole that shouldn't be there.\n\nLet's put this together: if a blood clot from the leg somehow travels to the l

### finetune setup

In [18]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/98.0 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

### medical reasoning gpt-oss-20b

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True
)

model_id = "dousery/medical-reasoning-gpt-oss-20b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=quantization_config,
    device_map="auto"
)

prompt = "A patient has symptoms of fever and cough. What could be the diagnosis?"
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

outputs = model.generate(
    **inputs,
    max_new_tokens=512,
    temperature=0.7,
    do_sample=True,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(generated_text)

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/27.9M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/446 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 3 files:   0%|          | 0/3 [00:00<?, ?it/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/2.53G [00:00<?, ?B/s]

model-00001-of-00003.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

### load curebench val dataset

In [ ]:
import pandas as pd
from typing import Dict, List
from torch.utils.data import DataLoader
from dataset_utils import build_dataset
from core.eval_framework import GPTOSS20BModel
from openai_harmony import load_harmony_encoding

In [ ]:
def load_dataset_by_config(config_path):
  # load config file to get dataset info
  config = json.load(open(config_path, 'r')) if config_path else {}
  if 'dataset' in config:
      dataset_config = config['dataset']
      dataset_name = dataset_config.get('dataset_name', 'treatment')
  print(f"\nconfig file: {config_path}\ncontents:\n{dataset_config}")
  dataset_path = dataset_config.get("dataset_path")

  # build dataset
  dataset = build_dataset(
        dataset_config.get("dataset_path"),
    )
  dataloader = DataLoader(dataset, batch_size=1, shuffle=False)
  dataset_list = []

  for batch in dataloader:
      question_type = batch[0][0]

      if question_type == "multi_choice":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
          })
      elif question_type == "open_ended_multi_choice":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
              "meta_question": batch[4][0],
          })
      elif question_type == "open_ended":
          dataset_list.append({
              "question_type": batch[0][0],
              "id": batch[1][0],
              "question": batch[2][0],
              "answer": batch[3][0],
          })
  return dataset_list

In [ ]:
val_data_config_path= "metadata_config_val.json"
val_data_list = load_dataset_by_config(val_data_config_path)


config file: metadata_config_val.json
contents:
{'dataset_name': 'cure_bench_phase1_val', 'dataset_path': 'resources/curebench_valset_pharse1.jsonl', 'description': 'CureBench 2025 val questions'}
dataset_path: resources/curebench_valset_pharse1.jsonl
CureBenchDataset initialized with 459 examples


In [ ]:
dataset = val_data_list[:10]

In [ ]:
dataset

[{'question_type': 'multi_choice',
  'id': 'U9PHZ83RKYV8',
  'question': 'Which drug brand name is associated with the treatment of acne?\nA: Salicylic Acid\nB: Minoxidil\nC: Ketoconazole\nD: Fluocinonide',
  'answer': 'A'},
 {'question_type': 'open_ended_multi_choice',
  'id': 'vIGwm8qguXYi',
  'question': 'What should patients do if they experience severe allergic reactions during or after receiving fosaprepitant for injection?',
  'answer': 'B',
  'meta_question': "The following is a multiple choice question about medicine and the agent's open-ended answer to the question. Convert the agent's answer to the final answer format using the corresponding option label, e.g., 'A', 'B', 'C', 'D', 'E' or 'None'. \n\nQuestion: What should patients do if they experience severe allergic reactions during or after receiving fosaprepitant for injection?\nA: Wait for the symptoms to resolve on their own.\nB: Inform their healthcare provider immediately and seek emergency medical care.\nC: Stop chem

In [ ]:
example = dataset[3]
print(example)

{'question_type': 'open_ended_multi_choice', 'id': 'WfWiWK0yULaX', 'question': 'Which of the following conditions is a contraindication for the use of Gadavist?', 'answer': 'B', 'meta_question': "The following is a multiple choice question about medicine and the agent's open-ended answer to the question. Convert the agent's answer to the final answer format using the corresponding option label, e.g., 'A', 'B', 'C', 'D', 'E' or 'None'. \n\nQuestion: Which of the following conditions is a contraindication for the use of Gadavist?\nA: Mild hypersensitivity reactions to Gadavist\nB: History of severe hypersensitivity reactions to Gadavist\nC: Renal impairment\nD: Liver dysfunction\n\n"}


In [ ]:
question = example["question"]
question_type = example["question_type"]

In [ ]:
# Format prompt
if question_type == "multi_choice":
    prompt = f"The following is a multiple choice question about medicine. Answer with a valid json containing the letter corresponding to the correct answer.\n\nQuestion: {question}\n\nAnswer:"
elif question_type == "open_ended_multi_choice" or question_type == "open_ended":
    prompt = f"The following is an open-ended question about medicine. Provide a comprehensive answer.\n\nQuestion: {question}\n\nAnswer:"

In [ ]:
print(f"prompt: {prompt}")

prompt: The following is an open-ended question about medicine. Provide a comprehensive answer.

Question: Which of the following conditions is a contraindication for the use of Gadavist?

Answer:


In [ ]:
system_identity = """
You are an expert medical assistant specializing in answering questions.

**Your communication MUST strictly adhere to the Harmony channels:**
1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.
2.  **final:** Use this channel for the final output intended for the user.

**Output Rule is Conditional:**
* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.
    * **Format:** `{"answer": "<LETTER>"}` (e.g., `{"answer": "A"}`)
* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coherent narrative response.

**Instruction:** Generate a complete response. The final output must use the 'final' channel and adhere to the conditional format rule."""

In [ ]:
# model = kit.model
model = GPTOSS20BModel("openai/gpt-oss-20b", reasoning_lvl="low", quantization="auto",
                       system_identity=system_identity)
model.load()

In [ ]:
# Pass this list to the stop_tokens argument
stop_sequences = [
    # 1. Stops the loop of internal reasoning tags
    "assistantfinal",

    # 2. Stops the common repetitive noise (often necessary)
    "analysisdone."
]

In [ ]:
response, reasoning_trace = model.inference(prompt,
    stop_strings=stop_sequences )

In [ ]:
response

'**Gadavist (gadobutrol)** is a gadolinium‑based contrast agent commonly used for magnetic resonance imaging (MRI) of the brain and spine. The safety of Gadavist, like all gadolinium‑based contrast agents, is contingent upon adequate renal clearance. **Severe or advanced renal impairment is the principal contraindication** because it predisposes patients to the rare but serious condition known as **Nephrogenic Systemic Fibrosis (NSF)** and increases the risk of gadolinium deposition.\n\nKey points:\n\n| Condition | Why it’s a contraindication for Gadavist | Typical guidelines |\n|-----------|------------------------------------------|--------------------|\n| **Severe chronic kidney disease (CKD Stage 4–5)** – eGFR < 30\u202fmL/min/1.73\u202fm² or requiring dialysis | Reduced elimination of gadobutrol → higher risk of NSF and gadolinium‑related toxicity | Avoid unless essential; if needed, use the lowest effective dose and consider alternative imaging modalities |\n| **Acute kidney inju

In [ ]:
reasoning_trace

[{'role': <Role.ASSISTANT: 'assistant'>,
  'name': None,
  'content': [{'type': 'text',
    'text': 'Answer is open-ended? They phrased "Which of the following conditions is a contraindication for the use of Gadavist?" But no options given. Likely expecting mention kidney disease (renal impairment). So provide narrative.'}],
  'channel': 'analysis'},
 {'role': <Role.ASSISTANT: 'assistant'>,
  'name': None,
  'content': [{'type': 'text',
    'text': '**Gadavist (gadobutrol)** is a gadolinium‑based contrast agent commonly used for magnetic resonance imaging (MRI) of the brain and spine. The safety of Gadavist, like all gadolinium‑based contrast agents, is contingent upon adequate renal clearance. **Severe or advanced renal impairment is the principal contraindication** because it predisposes patients to the rare but serious condition known as **Nephrogenic Systemic Fibrosis (NSF)** and increases the risk of gadolinium deposition.\n\nKey points:\n\n| Condition | Why it’s a contraindicatio

In [ ]:
print(model.system_identity)


You are an expert medical assistant specializing in answering questions.

**Your communication MUST strictly adhere to the Harmony channels:**
1.  **analysis:** Use this for all internal Chain-of-Thought (CoT), clinical reasoning, and factual processing. This content is for internal use only.
2.  **final:** Use this channel for the final output intended for the user.

**Output Rule is Conditional:**
* **If the input is a Multiple-Choice Question (MCQ):** Your output MUST be a single, valid JSON object containing only the selected answer letter.
    * **Format:** `{"answer": "<LETTER>"}` (e.g., `{"answer": "A"}`)
* **If the input is an Open-Ended Question:** Your output MUST be a detailed, coherent narrative response.

**Instruction:** Generate a complete response. The final output must use the 'final' channel and adhere to the conditional format rule.


### write submission to file

In [ ]:
# Generate submission with metadata from config/args
print("Generating submission with metadata...")
submission_path = kit.save_submission_with_metadata(
    results=[results],
    filename="submission.csv",
    config_path=config_path,
    args=""
)

print(f"\n✅ Evaluation completed successfully!")
print(f"📊 Accuracy: {results.accuracy:.2%} ({results.correct_predictions}/{results.total_examples})")
print(f"📄 Submission saved to: {submission_path}")

# # Show metadata summary if verbose
# final_metadata = kit.get_metadata(getattr(args, 'config', None), args)
# print("\n📋 Final metadata:")
# for key, value in final_metadata.items():
#     print(f"  {key}: {value}")


### stopping criteria

In [ ]:
from transformers import StoppingCriteria, StoppingCriteriaList

In [ ]:
stopping_criteria = StoppingCriteriaList()

In [ ]:
StoppingCriteria

transformers.generation.stopping_criteria.StoppingCriteria

In [ ]:
stopping_criteria

[]

In [ ]:
kit.model.enc